In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bangla-parts-of-speechpos-tag/postag.txt
/kaggle/input/subjectivity-detection-splitted/Split 70/test.xlsx
/kaggle/input/subjectivity-detection-splitted/Split 70/train.xlsx
/kaggle/input/subjectivity-detection-splitted/Split 70/val.xlsx
/kaggle/input/subjectivity-detection-splitted/Split80/test.xlsx
/kaggle/input/subjectivity-detection-splitted/Split80/train.xlsx
/kaggle/input/subjectivity-detection-splitted/Split80/val.xlsx


In [2]:
#!pip install bnlp-toolkit --quiet

In [3]:
'''from bnlp import BengaliPOS

bn_pos = BengaliPOS()

text = "আমরা নিজেরা যারা এর তৈরিতে যুক্ত ছিলাম" # or you can pass ['আমি', 'ভাত', 'খাই', '।']
res = bn_pos.tag(text)
print(res)
# [('আমি', 'PPR'), ('ভাত', 'NC'), ('খাই', 'VM'), ('।', 'PU')] '''

'from bnlp import BengaliPOS\n\nbn_pos = BengaliPOS()\n\ntext = "আমরা নিজেরা যারা এর তৈরিতে যুক্ত ছিলাম" # or you can pass [\'আমি\', \'ভাত\', \'খাই\', \'।\']\nres = bn_pos.tag(text)\nprint(res)\n# [(\'আমি\', \'PPR\'), (\'ভাত\', \'NC\'), (\'খাই\', \'VM\'), (\'।\', \'PU\')] '

In [4]:
#config
RANDOM_SEED = 42
import random
import os
import torch

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True  # Slower but reproducible
    torch.backends.cudnn.benchmark = False     # Disable benchmarking to ensure determinism

    # Set seed in Transformers (for Trainer)
    from transformers import set_seed as hf_set_seed
    hf_set_seed(seed)

set_seed(42)

2025-06-13 12:58:40.907788: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749819521.119621      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749819521.183837      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
'''
train_df = pd.read_excel("/kaggle/input/subjectivity-detection-splitted/Split 70/train.xlsx", engine='openpyxl')
val_df   = pd.read_excel("/kaggle/input/subjectivity-detection-splitted/Split 70/val.xlsx", engine='openpyxl')
test_df  = pd.read_excel("/kaggle/input/subjectivity-detection-splitted/Split 70/test.xlsx", engine='openpyxl')
'''
train_df = pd.read_excel("/kaggle/input/subjectivity-detection-splitted/Split80/train.xlsx", engine='openpyxl')
val_df   = pd.read_excel("/kaggle/input/subjectivity-detection-splitted/Split80/val.xlsx", engine='openpyxl')
test_df  = pd.read_excel("/kaggle/input/subjectivity-detection-splitted/Split80/test.xlsx", engine='openpyxl')


In [ ]:
from sklearn.preprocessing import LabelEncoder

def encode_labels(train_df, val_df, test_df):
    """
    Encodes the 'Label' column in train, val, and test DataFrames using LabelEncoder.
    Returns the updated DataFrames.
    """
    le = LabelEncoder()
    train_df['Label'] = le.fit_transform(train_df['Label'])
    val_df['Label']   = le.transform(val_df['Label'])
    test_df['Label']  = le.transform(test_df['Label'])
    return train_df, val_df, test_df


def prepare_datasets(train_df, val_df, test_df, transformerDL=True):
    """
    Extracts features and labels from the given DataFrames based on transformerDL flag.
    Returns: X_train, X_val, X_test, y_train, y_val, y_test
    """
    text_col = 'Text' if transformerDL else 'cleanedText'
    
    X_train = train_df[text_col]
    y_train = train_df['Label']
    
    X_val = val_df[text_col]
    y_val = val_df['Label']
    
    X_test = test_df[text_col]
    y_test = test_df['Label']
    
    return X_train, X_val, X_test, y_train, y_val, y_test
    
# Encode labels
train_df, val_df, test_df = encode_labels(train_df, val_df, test_df)

# Prepare datasets
X_train, X_val, X_test, y_train, y_val, y_test = prepare_datasets(train_df, val_df, test_df, transformerDL=True)

import wandb
wandb.login(key=wandb_key)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: minhazulkabir (minhazulkabir-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, f1_score, classification_report
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import shutil
import gc

#models
model_configs = [
    {"name": "buetBangla-BERT", "model_name": "csebuetnlp/banglabert"},
    {"name": "sagorBangla-BERT", "model_name": "sagorsarker/bangla-bert-base"},
    {"name": "Multilingual BERT", "model_name": "bert-base-multilingual-cased"},
    {"name": "XLM-Roberta", "model_name": "xlm-roberta-base"},
    {"name": "DistilBERT Multilingual", "model_name": "distilbert-base-multilingual-cased"},
    {"name": "IndicBERT", "model_name": "ai4bharat/indic-bert"},
    {"name": "MuRIL", "model_name": "google/muril-base-cased"},
    #{"name": "Bangla Electra", "model_name": "monsoon-nlp/bangla-electra"},
]

#Dataset
class BanglaDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        item = {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
        }
        if "token_type_ids" in encoding:
            item["token_type_ids"] = encoding["token_type_ids"].squeeze(0)
        if self.labels is not None:
            item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

#Evaluation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    if isinstance(logits, tuple):
        logits = logits[0]
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="macro"),
    }
#param
def print_model_summary(model):
    print("📄 Model Architecture:\n")
    print(model)

    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    print(f"\n🔢 Total parameters: {total_params}")
    print(f"🛠️ Trainable parameters: {trainable_params}")

#Loop through models
for config in model_configs:
    print(f"\n Training & Evaluating: {config['name']}")
    
    tokenizer = AutoTokenizer.from_pretrained(config["model_name"])
    model = AutoModelForSequenceClassification.from_pretrained(config["model_name"], num_labels=2)
    print_model_summary(model)
    #Build dataset
    train_dataset = BanglaDataset(X_train.tolist(), y_train.tolist(), tokenizer, max_len=128)
    val_dataset = BanglaDataset(X_val.tolist(), y_val.tolist(), tokenizer, max_len=128)
    test_dataset = BanglaDataset(X_test.tolist(), y_test.tolist(), tokenizer, max_len=128)

    #Training arguments
    training_args = TrainingArguments(
        output_dir=f"./results/{config['name'].replace(' ', '_')}",
        num_train_epochs=15,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        logging_dir=f"./logs/{config['name'].replace(' ', '_')}",
        logging_steps=100,
        eval_strategy="steps",
        eval_steps=300,
        save_strategy="steps",
        save_steps=300,
        save_total_limit=1,
        load_best_model_at_end=True,
        metric_for_best_model='f1',
        greater_is_better=True,
        weight_decay=0.01,
        learning_rate=1e-5,
        lr_scheduler_type="linear",
        warmup_ratio=0.1,
        fp16=True,
        report_to=None
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    )

    trainer.train()

    #Evaluate on validation
    print(f"\n📊 Validation Results for {config['name']}:")
    val_results = trainer.evaluate()
    print(val_results)

    #Evaluate on test
    print(f"\n📦 Evaluating on Test Set...")
    test_results = trainer.evaluate(test_dataset)
    print(f"📊 Test Results: {test_results}")

    #Classification report
    predictions = trainer.predict(test_dataset)
    y_pred = np.argmax(predictions.predictions, axis=1)
    y_true = predictions.label_ids
    print(f"\n📄 Classification Report for {config['name']}:\n")
    print(classification_report(y_true, y_pred))

    # free memory and delete checkpoints
    del model, tokenizer, trainer
    torch.cuda.empty_cache()
    gc.collect()
    shutil.rmtree(f"./results/{config['name'].replace(' ', '_')}", ignore_errors=True)



 Training & Evaluating: buetBangla-BERT


tokenizer_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/586 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/528k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

📄 Model Architecture:

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(197285, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): 

A ConfigError was raised whilst setting the number of model parameters in Weights & Biases config.


Step,Training Loss,Validation Loss,Accuracy,F1
300,0.651200,0.628740,0.579191,0.366764
600,0.448200,0.423871,0.875145,0.873620
900,0.333000,0.327421,0.891329,0.890059
1200,0.250000,0.319679,0.899422,0.895381
1500,0.197100,0.331650,0.895954,0.891925
1800,0.130500,0.381003,0.889017,0.887719



📊 Validation Results for MuRIL:


{'eval_loss': 0.31967881321907043, 'eval_accuracy': 0.8994219653179191, 'eval_f1': 0.8953810864351996, 'eval_runtime': 3.3177, 'eval_samples_per_second': 260.723, 'eval_steps_per_second': 16.578, 'epoch': 4.157043879907621}

📦 Evaluating on Test Set...
📊 Test Results: {'eval_loss': 0.2502436935901642, 'eval_accuracy': 0.9237875288683602, 'eval_f1': 0.9208803986710964, 'eval_runtime': 3.3289, 'eval_samples_per_second': 260.145, 'eval_steps_per_second': 16.522, 'epoch': 4.157043879907621}

📄 Classification Report for MuRIL:

              precision    recall  f1-score   support

           0       0.91      0.96      0.94       501
           1       0.95      0.87      0.91       365

    accuracy                           0.92       866
   macro avg       0.93      0.92      0.92       866
weighted avg       0.93      0.92      0.92       866

